# Jupyter notebook for quick analysis of WNV climate input data

In [1]:
import xarray as xr
import pickle as pk
import time
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
import mapclassify as mc
from copy import deepcopy as cp
import os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy as cr
import geopandas as gpd
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline


Define functions

In [2]:
def get_seasons(
    da,
    month_length,
):
    return ((da * month_length).resample(time='QS-DEC').sum() / month_length.resample(time='QS-DEC').sum())
    

Set up dask client

In [3]:
import dask.multiprocessing
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(dashboard_address='12345', n_workers=2, threads_per_worker=6, local_directory='/vscmnt/brussel_pixiu_data/_data_brussel/vo/000/bvo00012/vsc10116/lifetime_exposure_isimip')
client = Client(cluster)
client
from pprint import pprint
pprint(cluster.worker_spec)
print(client)

2023-03-30 09:56:49,194 - distributed.diskutils - INFO - Found stale lock file and directory '/vscmnt/brussel_pixiu_data/_data_brussel/vo/000/bvo00012/vsc10116/lifetime_exposure_isimip/dask-worker-space/worker-x7xbl_im', purging
2023-03-30 09:56:49,228 - distributed.diskutils - INFO - Found stale lock file and directory '/vscmnt/brussel_pixiu_data/_data_brussel/vo/000/bvo00012/vsc10116/lifetime_exposure_isimip/dask-worker-space/worker-yd4cmjn8', purging


{0: {'cls': <class 'distributed.nanny.Nanny'>,
     'options': {'dashboard': False,
                 'dashboard_address': None,
                 'host': '127.0.0.1',
                 'interface': None,
                 'local_directory': '/vscmnt/brussel_pixiu_data/_data_brussel/vo/000/bvo00012/vsc10116/lifetime_exposure_isimip',
                 'memory_limit': 16106127360,
                 'nthreads': 6,
                 'protocol': 'tcp://',
                 'security': Security(require_encryption=False, tls_min_version=771),
                 'services': {},
                 'silence_logs': 30}},
 1: {'cls': <class 'distributed.nanny.Nanny'>,
     'options': {'dashboard': False,
                 'dashboard_address': None,
                 'host': '127.0.0.1',
                 'interface': None,
                 'local_directory': '/vscmnt/brussel_pixiu_data/_data_brussel/vo/000/bvo00012/vsc10116/lifetime_exposure_isimip',
                 'memory_limit': 16106127360,
               

In [4]:
os.chdir('/vscmnt/brussel_pixiu_data/_data_brussel/vo/000/bvo00012/vsc10116/wnv')
scriptsdir = os.getcwd()
pkdir = os.path.join(scriptsdir,'pickles')
figdir = os.path.join(scriptsdir,'figures')

In [5]:
print(scriptsdir)
print(pkdir)
print(figdir)

/vscmnt/brussel_pixiu_data/_data_brussel/vo/000/bvo00012/vsc10116/wnv
/vscmnt/brussel_pixiu_data/_data_brussel/vo/000/bvo00012/vsc10116/wnv/pickles
/vscmnt/brussel_pixiu_data/_data_brussel/vo/000/bvo00012/vsc10116/wnv/figures


Run factors

In [6]:
forcings=[
    'obsclim',
    'counterclim',
]
reanalysis_products=[
    '20CRv3',
    '20CRv3-ERA5',
    '20CRv3-W5E5',
    'GSWP3-W5E5',
]
variables=[
    'tas',
    'pr',
    'hurs',
]

Analyze

In [7]:
for f in forcings:
    for r in reanalysis_products:
        datadir='/vscmnt/brussel_pixiu_data/_data_brussel/vo/000/bvo00012/data/dataset/ISIMIP/ISIMIP3a/InputData/climate/atmosphere/{}/global/daily/historical/{}'.format(f,r)
        os.chdir(datadir)
        for v in variables:
            if not os.path.isfile(os.path.join(pkdir,'season_means_{}_{}_{}.pkl'.format(f,r,v))):
                start_time = time.time()
                files = [file for file in sorted(os.listdir(datadir)) if '_{}_'.format(v) in file and '.nc' in file]
                concat_list = []
                for file in files:
                    ds = xr.open_mfdataset(
                        file,
                        chunks={
                            'lat':60,
                            'lon':120,
                            'time':-1,
                        },
                        decode_times=False
                    )
                    da = ds[v].squeeze()
                    units, reference_date = da.time.attrs['units'].split('since')
                    y1=int(file.split('_')[-2])
                    reference_date = reference_date.replace(reference_date[1:5],str(y1))[1:]
                    new_date = pd.date_range(start=reference_date, periods=da.sizes['time'], freq='D')
                    da['time'] = new_date     
                    concat_list.append(da)
                da_all = xr.concat(concat_list,dim='time')
                da_all = da_all.chunk({'time':-1})
                da_month_length = da_all.time.dt.days_in_month
                da_season_labels=da_all.resample(time='QS-DEC').sum().time.data
                da_template = xr.DataArray(
                    data=np.full(
                        (len(da_season_labels),len(da_all.lat.data),len(da_all.lon.data)),
                        fill_value=np.nan,
                    ),
                    coords={
                        'time': ('time', da_season_labels),
                        'lat': ('lat', da_all.lat.data),
                        'lon': ('lon', da_all.lon.data),
                    }
                ).chunk({'time':-1,'lat':60,'lon':120})
                da_seasonal_means = xr.map_blocks(
                    get_seasons,
                    da_all,
                    args=[da_month_length],
                    template=da_template,
                ).load()
                with open(os.path.join(pkdir,'season_means_{}_{}_{}.pkl'.format(f,r,v)), 'wb') as f:
                    pk.dump(da_seasonal_means,f)   
                print("--- {} minutes for computing one dataset---".format(np.floor((time.time() - start_time) / 60)))                       
            else:
                pass
            

HDF5-DIAG: Error detected in HDF5 (1.12.2) MPI-process 0:
  #000: H5A.c line 528 in H5Aopen_by_name(): can't open attribute
    major: Attribute
    minor: Can't open object
  #001: H5VLcallback.c line 1091 in H5VL_attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #002: H5VLcallback.c line 1058 in H5VL__attr_open(): attribute open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: H5VLnative_attr.c line 130 in H5VL__native_attr_open(): can't open attribute
    major: Attribute
    minor: Can't open object
  #004: H5Aint.c line 545 in H5A__open_by_name(): unable to load attribute info from object header
    major: Attribute
    minor: Unable to initialize object
  #005: H5Oattribute.c line 494 in H5O__attr_open_by_name(): can't locate attribute: '_QuantizeBitGroomNumberOfSignificantDigits'
    major: Attribute
    minor: Object not found
HDF5-DIAG: Error detected in HDF5 (1.12.2) MPI-process 0:
  #000: H5A.c line 5

--- 49.0 minutes for computing one dataset---


FileNotFoundError: [Errno 2] No such file or directory: "/vscmnt/brussel_pixiu_data/_data_brussel/vo/000/bvo00012/data/dataset/ISIMIP/ISIMIP3a/InputData/climate/atmosphere/<_io.BufferedWriter name='/vscmnt/brussel_pixiu_data/_data_brussel/vo/000/bvo00012/vsc10116/wnv/pickles/season_means_obsclim_20CRv3_hurs.pkl'>/global/daily/historical/20CRv3-ERA5"